# Model
Ce programme Résout le FJSSP avec optimisation des tâches de maintenance tout en respectant des contraintes de qualité minimale

In [1]:
from my_module import create_gantt_chart, parse_operations_file, parse_degradations_file
from mip import Model, xsum, maximize, minimize, BINARY
import numpy as np
import matplotlib.pyplot as plt
import json

C:\Users\useer\Desktop\STAGE CESI\Réalisations\Versions\my_module.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\useer\Desktop\STAGE CESI\Réalisations\Versions\my_module.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
model = Model()

# Extraction des données et définition des constants

In [3]:
inf= 999999
instancename = 'sfjs07'
n_max = 3
num_instance = 'instance02'
filename = f"../TESTS/{instancename}/{instancename}.txt"
nbJobs, nbMachines, nbOperationsParJob, dureeOperations = parse_operations_file(filename)
filename = f"../TESTS/{instancename}/{num_instance}/instance.txt"
_, _, nbComposants, seuils_degradation, dureeMaintenances, degradations = parse_degradations_file(filename)

alpha_kl = [[.01 for l in range(nbComposants[k])] for k in range(nbMachines)]
AQL = 0.05
Qjmin = [0.8 for j in range(nbJobs)]
couleurMachines = {
    'J1': '#ff9999',   # Light Red
    'J2': '#99ff99',   # Light Green
    'J3': '#9999ff',   # Light Blue
    'J4': '#61620abf', # Dark Blue
    'J5': '#ffcc99',   # Light Orange
    'J6': '#66c2a5',   # Teal
    'J7': '#fc8d62',   # Salmon
    'J8': '#8da0cb',   # Lavender
    'J9': '#e78ac3',   # Pink
    'J10': '#a6d854',  # Lime Green
    'J11': '#ffd92f',  # Yellow
    'J12': '#e5c494',  # Beige
    'J13': '#b3b3b3',  # Grey
    'J14': '#1f78b4',  # Bright Blue
    'J15': '#33a02c',  # Bright Green
    'Maintenances': '#2F4F4F'  # Dark Slate Grey
}

# Définfition des Variables

In [4]:
maxComposants = max(nbComposants)
D_max = t_max = M = 999
Cmax = model.add_var()
Mmax = model.add_var()
x_ijkn = [[[[model.add_var(var_type=BINARY) for i in range(nbOperationsParJob[j])] for j in range(nbJobs)] for k in range(nbMachines)] for n in range(n_max)]
y_kln = [[[model.add_var(var_type=BINARY) for l in range(nbComposants[k])] for k in range(nbMachines)] for n in range(n_max)]
v_kn= [[model.add_var() for k in range(nbMachines)] for n in range(n_max)]
w = [[[[[[model.add_var(var_type=BINARY) for i in range(nbOperationsParJob[j])] for j in range(nbJobs)] for i_ in range(nbOperationsParJob[j_])] for j_ in range(nbJobs)] for m in range(nbMachines)] for n in range(n_max)]
z = [[[[[[[model.add_var(var_type=BINARY) for i in range(nbOperationsParJob[j])] for j in range(nbJobs)] for i_ in range(nbOperationsParJob[j_])] for j_ in range(nbJobs)] for l in range(maxComposants)] for k in range(nbMachines)] for n in range(n_max)]
d_kln = [[[model.add_var() for l in range(nbComposants[k])] for k in range(nbMachines)] for n in range(n_max)]
prod_y_d = [[[model.add_var() for l in range(nbComposants[k])] for k in range(nbMachines)] for n in range(n_max)]
prod_w_t = [[[[[[model.add_var() for i in range(nbOperationsParJob[j])] for j in range(nbJobs)] for i_ in range(nbOperationsParJob[j_])] for j_ in range(nbJobs)] for m in range(nbMachines)] for n in range(n_max)]
D_kln = [[[model.add_var() for l in range(nbComposants[k])] for k in range(nbMachines)] for n in range(n_max)]
t_ij = [[model.add_var() for i in range(nbOperationsParJob[j])] for j in range(nbJobs)]
Qinitj = [1.0 for j in range(nbJobs)]  # Exemple de taux de qualité initiale
Qjmin = [0.8 for j in range(nbJobs)]   # Exemple de taux de qualité minimale acceptable
alpha_kl = [[.01 for l in range(nbComposants[k])] for k in range(nbMachines)]  # Exemple de coefficients de détérioration de la qualité   
prod_x_D = [[[[[model.add_var() for i in range(nbOperationsParJob[j])] for j in range(nbJobs)] for l in range(nbComposants[k])] for k in range(nbMachines)] for n in range(n_max)]
Qj = [model.add_var()  for j in range(nbJobs)]
prod_x_y = [[[[[model.add_var(var_type=BINARY) for i in range(nbOperationsParJob[j])] for j in range(nbJobs)] for l in range(nbComposants[k])] for k in range(nbMachines)] for n in range(n_max)]

# Calcul de la Dégradation

In [5]:
for n in range(n_max):
    for k in range(nbMachines):
        for l in range(nbComposants[k]):
             model += d_kln[n][k][l] == xsum(x_ijkn[n][k][j][i] * degradations[k][l][j][i]
                            for j in range(nbJobs)
                            for i in range(nbOperationsParJob[j]))
            
for k in range(nbMachines):
        for l in range(nbComposants[k]):
            model += D_kln[0][k][l] ==  d_kln[0][k][l]
for n in range(n_max-1):
    for k in range(nbMachines):
        for l in range(nbComposants[k]):
            model += D_kln[n+1][k][l] == D_kln[n][k][l] - prod_y_d[n][k][l] + d_kln[n+1][k][l]

## prod_y_d[n-1][k][l] = y_kln[n-1][k][l]*D_kln[n-1][k][l]
for n in range(n_max):
    for k in range(nbMachines):
        for l in range(nbComposants[k]):
            model += prod_y_d[n][k][l] >= 0
            model += prod_y_d[n][k][l] - y_kln[n][k][l] * D_max <= 0
            model += prod_y_d[n][k][l] - D_kln[n][k][l] <= 0
            model += prod_y_d[n][k][l] - D_kln[n][k][l] + D_max - y_kln[n][k][l]*D_max >= 0

# Calcul de la Détérioration De La Qualité

In [6]:
for n in range(n_max) :
    if n != 0:
        for k in range(nbMachines) :
            for l in range(nbComposants[k]) :
                for j in range(nbJobs) :
                    for i in range(nbOperationsParJob[j]) :
                        model += prod_x_y[n][k][l][j][i] - x_ijkn[n][k][j][i] + y_kln[n-1][k][l] >= 0
                        model += prod_x_y[n][k][l][j][i] - (1/2)*x_ijkn[n][k][j][i] + (1/2)*y_kln[n-1][k][l] <= (1/2)
for n in range(n_max):
    if n != 0:
        for k in range(nbMachines):
            for l in range(nbComposants[k]):
                for j in range(nbJobs):
                    for i in range(nbOperationsParJob[j]):
                        model += prod_x_D[n][k][l][j][i] >= 0
                        model += prod_x_D[n][k][l][j][i] - D_max * prod_x_y[n][k][l][j][i] <= 0
                        model += prod_x_D[n][k][l][j][i] - D_kln[n-1][k][l] <= 0
                        model += prod_x_D[n][k][l][j][i] - D_kln[n-1][k][l] + D_max - prod_x_y[n][k][l][j][i]*D_max >= 0
    else:
        for k in range(nbMachines):
            for l in range(nbComposants[k]):
                for j in range(nbJobs):
                    for i in range(nbOperationsParJob[j]):
                        model += prod_x_D[n][k][l][j][i] == 0

for j in range(nbJobs):
    degradation_sum = xsum(prod_x_D[n][k][l][j][i]  * alpha_kl[k][l] 
                           for i in range(nbOperationsParJob[j]) 
                           for n in range(n_max) 
                           for k in range(nbMachines) 
                           for l in range(nbComposants[k]))
    model += Qj[j] == Qinitj[j] - degradation_sum

# Calcul des variables

In [7]:
## CALCUL DE LA VARIABLE y
for n in range(n_max):
    for k in range(nbMachines):
        for l in range(nbComposants[k]):
            model += M*y_kln[n][k][l] - D_kln[n][k][l] + seuils_degradation[k][l]>= 0

## CALCUL DE LA VARIABLE v
for n in range(n_max):
    for k in range(nbMachines):
        model += v_kn[n][k] == xsum(x_ijkn[n][k][j][i] 
                                    for j in range(nbJobs)
                                    for i in range(nbOperationsParJob[j]))

## CALCUL DE LA VARIABLE w
for n in range(n_max-1):
    for k in range(nbMachines):
        for j_ in range(nbJobs):
            for i_ in range(nbOperationsParJob[j_]):
                for j in range(nbJobs):
                    for i in range(nbOperationsParJob[j]):
                        model += w[n][k][j_][i_][j][i] - x_ijkn[n][k][j][i] - x_ijkn[n+1][k][j_][i_] >= -1
                        model += w[n][k][j_][i_][j][i] - 0.5*x_ijkn[n][k][j][i] - 0.5*x_ijkn[n+1][k][j_][i_] <= 0

## CALCUL DE LA VARIABLE z
for n in range(n_max-1):
    for k in range(nbMachines):
        for j_ in range(nbJobs):
            for i_ in range(nbOperationsParJob[j_]):
                for j in range(nbJobs):
                    for i in range(nbOperationsParJob[j]):
                        for l in range(nbComposants[k]):
                            model +=  z[n][k][l][j_][i_][j][i] - w[n][k][j_][i_][j][i] - y_kln[n][k][l] >= -1
                            model +=  z[n][k][l][j_][i_][j][i] - 0.5*w[n][k][j_][i_][j][i] - 0.5*y_kln[n][k][l] <= 0

## LINEARISATION DE w ET DE tij
for n in range(n_max):
    for k in range(nbMachines):
        for j_ in range(nbJobs):
            for i_ in range(nbOperationsParJob[j_]):
                for j in range(nbJobs):
                    for i in range(nbOperationsParJob[j]):
                        model += prod_w_t[n][k][j_][i_][j][i] >= 0
                        model += prod_w_t[n][k][j_][i_][j][i] - w[n][k][j_][i_][j][i] * t_max <= 0
                        model += prod_w_t[n][k][j_][i_][j][i] - t_ij[j][i] <= 0
                        model += prod_w_t[n][k][j_][i_][j][i] - t_ij[j][i] + t_max - w[n][k][j_][i_][j][i]*t_max >= 0

# Contraintes

In [8]:
## CONTRAINTES DE PRECEDENCE DES OPERATIONS
for j in range(nbJobs):
    for i in range(nbOperationsParJob[j] - 1):
            model += t_ij[j][i+1] >= t_ij[j][i] + xsum(x_ijkn[n][k][j][i] * dureeOperations[k][j][i]
                                                       for n in range(n_max)
                                                       for k in range(nbMachines))

## CONTRAINTES DE NON CHEVAUVHEMENT DES OPERATIONS SUR LES MACHINES
for l in range(maxComposants):
    for j_ in range(nbJobs):
        for i_ in range(nbOperationsParJob[j_]):
            model += t_ij[j_][i_] >=  xsum(prod_w_t[n][k][j_][i_][j][i] + w[n][k][j_][i_][j][i] * dureeOperations[k][j][i] +  z[n][k][l][j_][i_][j][i] * dureeMaintenances[k][l]
                                           for n in range(n_max)
                                           for k in range(nbMachines)
                                           for j in range(nbJobs)
                                           for i in range(nbOperationsParJob[j]))

## CONTRAINTES POUR IMPOSER QUE CHAQUE OPERATION SOIT ORDONNANCEE
for j in range(nbJobs):
    for i in range(nbOperationsParJob[j]):
        model += xsum(x_ijkn[n][k][j][i] for n in range(n_max) for k in range(nbMachines)) == 1

## CONTRAINTES SUR LES INDCES n DES OPERATIONS DES MACHINES
for k in range(nbMachines):
    for n in range(n_max-1):
        model += v_kn[n][k] >= v_kn[n+1][k]
        model += v_kn[n][k] <= 1

## CONTRAINTES DE QUALITE
for j in range(nbJobs):
    model +=  xsum((1-Qj[j]) for j in range(nbJobs))/nbJobs <= AQL

# Fonction Objectif

In [9]:
for j in range(nbJobs):
    model += Cmax >= t_ij[j][nbOperationsParJob[j]-1] + xsum(x_ijkn[n][k][j][nbOperationsParJob[j]-1] * dureeOperations[k][j][nbOperationsParJob[j]-1] 
                                                             for n in range(n_max) 
                                                             for k in range(nbMachines))

model += Mmax >= xsum(y_kln[n][k][l] for n in range(n_max) 
                      for k in range(nbMachines) 
                      for l in range(nbComposants[k]))

In [10]:
model.objective = minimize(0.9*Cmax + 0.1*Mmax)
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [11]:
# Affichage des valeurs des variables de décision
print('###############################')
for j_ in range(nbJobs):
    for i_ in range(nbOperationsParJob[j_]):
        for n in range(n_max):
            for k in range(nbMachines):
                for j in range(nbJobs):
                    for i in range(nbOperationsParJob[j]):
                        if w[n][k][j_][i_][j][i].x != 0:
                            print(f"w[{n+1}][{k+1}][{j_+1}][{i_+1}][{j+1}][{i+1}] = {w[n][k][j_][i_][j][i].x}")
print('###############################')
for n in range(n_max):
    for k in range(nbMachines):
        for j in range(nbJobs):
            for i in range(nbOperationsParJob[j]):
                if x_ijkn[n][k][j][i].x != 0:
                    print(f"x_ijkn[{n+1}][{k+1}][{j+1}][{i+1}] = {x_ijkn[n][k][j][i].x}")             
print('###############################')
for n in range(n_max):
    for k in range(nbMachines):
        for l in range(nbComposants[k]):
            if y_kln[n][k][l].x != 0 :
                print(f"y_nkl[{n+1}][{k+1}][{l+1}] = {y_kln[n][k][l].x}")
print('###############################')
deg = [[] for k in range(nbMachines) for l in range(nbComposants[k])]
for n in range(n_max):
    ind = 0
    for k in range(nbMachines):
        for l in range(nbComposants[k]):
            deg[ind].append(D_kln[n][k][l].x)
            ind += 1
ind = 0 
for k in range(nbMachines):
    print(f"dégradations machine {k+1} :")
    for l in range(nbComposants[k]):
        deg[ind].insert(0, 0)
        print(deg[ind])
        ind += 1
print('###############################')
for j in range(nbJobs):
    for i in range(nbOperationsParJob[j]):
        print(f"t_ij[{j+1}][{i+1}] = {t_ij[j][i].x}")
print('###############################')
for n in range(n_max):
    for k in range(nbMachines):
        for l in range(nbComposants[k]):
            if y_kln[n][k][l].x == 1:
                debut = 0
                for j in range(nbJobs):
                    for i in range(nbOperationsParJob[j]):
                        if x_ijkn[n][k][j][i].x == 1:
                            print(f"maintenance machine:{k+1}, 'composant':{l+1}, 'debut':{t_ij[j][i].x + dureeOperations[k][j][i]}, 'fin':{debut+dureeMaintenances[k][l]}")
print('###############################')
for j in range(nbJobs):
    print(f"Qj[{j+1}] = {Qj[j].x}")

###############################
w[1][2][1][2][3][1] = 1.0
w[1][4][1][3][3][2] = 1.0
w[2][2][2][2][1][2] = 1.0
w[1][5][2][3][3][3] = 1.0
###############################
x_ijkn[1][1][1][1] = 1.0
x_ijkn[1][2][3][1] = 1.0
x_ijkn[1][3][2][1] = 1.0
x_ijkn[1][4][3][2] = 1.0
x_ijkn[1][5][3][3] = 1.0
x_ijkn[2][2][1][2] = 1.0
x_ijkn[2][4][1][3] = 1.0
x_ijkn[2][5][2][3] = 1.0
x_ijkn[3][2][2][2] = 1.0
###############################
y_nkl[1][1][1] = 1.0
y_nkl[1][3][1] = 1.0
y_nkl[1][4][1] = 1.0
y_nkl[1][5][1] = 1.0
y_nkl[2][2][1] = 1.0
y_nkl[2][5][1] = 1.0
###############################
dégradations machine 1 :
[0, 12.0, 0.0, 0.0]
dégradations machine 2 :
[0, 9.0, 19.0, 14.0]
dégradations machine 3 :
[0, 15.0, 0.0, 0.0]
dégradations machine 4 :
[0, 8.0, 9.0, 9.0]
dégradations machine 5 :
[0, 15.0, 12.0, 0.0]
###############################
t_ij[1][1] = 0.0
t_ij[1][2] = 117.0
t_ij[1][3] = 247.0
t_ij[2][1] = 0.0
t_ij[2][2] = 252.0
t_ij[2][3] = 318.0
t_ij[3][1] = 0.0
t_ij[3][2] = 62.0
t_ij[3][3] = 1

In [12]:
def creerDiagrammeGantt(dureeOperations, couleurMachines):
    taches = []
    for j in range(nbJobs):
        for i in range(nbOperationsParJob[j]):
            for n in range(n_max):
                for k in range(nbMachines):
                    if x_ijkn[n][k][j][i].x == 1:
                        machine = k+1
            start_time = t_ij[j][i].x
            finish_time = start_time + dureeOperations[machine-1][j][i]
            tache = dict(task=f"Machine {machine}",
                         start=start_time,
                         end=finish_time,
                         rsc=f"J{j+1}",
                         label=f"O{j+1}:{i+1}",
                         info=f"J{j+1}")
            taches.append(tache)
    for n in range(n_max):
        for k in range(nbMachines):
            for l in range(nbComposants[k]):
                if y_kln[n][k][l].x == 1:
                    debut = 0
                    for j in range(nbJobs):
                        for i in range(nbOperationsParJob[j]):
                            if x_ijkn[n][k][j][i].x == 1:
                                if n+1 != n_max :
                                    if xsum(x_ijkn[n+1][k][j_][i_] for j_ in range(nbJobs) for i_ in range(nbOperationsParJob[j_])).x != 0:
                                        machine = k+1
                                        composant = l+1
                                        start_time = t_ij[j][i].x + dureeOperations[k][j][i]
                                        finish_time = start_time + dureeMaintenances[k][l]
                                        tache = dict(task=f"Machine {machine}", 
                                                     start=start_time, 
                                                     end=finish_time, 
                                                     rsc=f"Maintenances",
                                                     label=f"M", 
                                                     info=f"Composant : {composant} (durée={dureeMaintenances[k][l]})")
                                        taches.append(tache)
    taches = sorted(taches, key=lambda x: x['task'])
    return taches

In [13]:
# # Choix de couleurs pour les seuils
# colors = ['r','b','y']

# # Tracer les courbes par machine
# fig, axes = plt.subplots(nbMachines, 1, figsize=(12, 16), sharex=True)

# ind = 0
# for k in range(nbMachines):
#     col = 0
#     for l in range(nbComposants[k]):
#         axes[k].plot(deg[ind], label=f'Composant {l+1}', color=colors[col])
#         axes[k].axhline(y=seuils_degradation[k][l], color=colors[col], linestyle='--', label=f'Seuil Composant {l+1}')
#         ind += 1
#         col += 1
#     axes[k].set_title(f'Machine {k+1}')
#     axes[k].set_ylabel('Dégradation')
#     axes[k].legend()
#     axes[k].grid(True)

# plt.xlabel('Cycle (n)')
# plt.tight_layout()
# plt.show()

print("Valeur Cmax : ", Cmax.x)
fig = creerDiagrammeGantt(dureeOperations, couleurMachines)
app = create_gantt_chart(couleurMachines, fig, title="Ordonnancement", cell_widht=1, cell_height=40)
if __name__ == '__main__':
    app.run_server(debug=True, port=8030)

data = {
    "Cmax_x": Cmax.x,
    "fig": fig,
    "quality": [elt.x for elt in Qj],
    "degradations": deg
}
print( sum([(1-Qj[j].x) for j in range(nbJobs)])/nbJobs)
      
# Écrire les données dans un fichier JSON
# with open('../TESTS/k1/instance01/test_result.json', 'w') as json_file:
#     json.dump(data, json_file, indent=4)

Valeur Cmax :  397.0


0.02999999999999999
